In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
import numpy as np

from PIL import Image
from PIL import ImageStat
from PIL import ImageOps
from torch_geometric.data import Data
from skimage.measure import regionprops
from histocartography.utils import download_example_data
from histocartography.preprocessing import (
    NucleiExtractor,
    DeepFeatureExtractor,
    KNNGraphBuilder
)
from histocartography.preprocessing import (
    ColorMergedSuperpixelExtractor,
    DeepFeatureExtractor,
    RAGGraphBuilder
)
import warnings
warnings.filterwarnings('ignore')

from histocartography.visualization import OverlayGraphVisualization
visualizer = OverlayGraphVisualization()

Using backend: pytorch


# Encapsulate CRCS dataset

In [2]:
nuclei_detector = NucleiExtractor()
feature_extractor_for_cell = DeepFeatureExtractor(
        architecture='resnet34',
        patch_size=72,
        resize_size=224
    )
knn_graph_builder = KNNGraphBuilder(k=5, thresh=50, add_loc_feats=True)

File already downloaded.


In [3]:
def return_cell_graph(image):
 
    image = np.array(image)
    nuclei_map, nuclei_centroids = nuclei_detector.process(image)
    
    image_size = (nuclei_map.shape[1], nuclei_map.shape[0])  # (x, y)

    features = feature_extractor_for_cell.process(image, nuclei_map)
    
    graph = knn_graph_builder.process(nuclei_map, features)
    
    canvas = visualizer.process(image, graph, instance_map=nuclei_map)   # image对象
#     canvas.show()
    
    return nuclei_map, nuclei_centroids, features,canvas

In [4]:
superpixel_detector_1 = ColorMergedSuperpixelExtractor(
        nr_superpixels=100,
        compactness=20,
        blur_kernel_size=1,
        threshold=0.001,
    )
superpixel_detector_2 = ColorMergedSuperpixelExtractor(
        nr_superpixels=200,
        compactness=20,
        blur_kernel_size=1,
        threshold=0.001,
    )
superpixel_detector_3 = ColorMergedSuperpixelExtractor(
        nr_superpixels=300,
        compactness=20,
        blur_kernel_size=1,
        threshold=0.001,
    )

feature_extractor_for_tissue = DeepFeatureExtractor(
        architecture='resnet34',
        patch_size=144,
        resize_size=224
    )
tissue_graph_builder = RAGGraphBuilder(add_loc_feats=True)

In [5]:
def return_tissue_graph_1(image):
    
    image = np.array(image)
    
    superpixels, _ = superpixel_detector_1.process(image)
    
    regions = regionprops(superpixels)
    centroids = np.empty((len(regions), 2))
    for i, region in enumerate(regions):
        center_y, center_x = region.centroid  # (y, x)
        center_x = int(round(center_x))
        center_y = int(round(center_y))
        centroids[i, 0] = center_x
        centroids[i, 1] = center_y
    
    features = feature_extractor_for_tissue.process(image, superpixels)
    
    graph = tissue_graph_builder.process(superpixels, features)
    
    canvas = visualizer.process(image, graph, instance_map=superpixels)
#     canvas.show()
    return superpixels,centroids,features,canvas


def return_tissue_graph_2(image):
    image = np.array(image)
    
    superpixels, _ = superpixel_detector_2.process(image)
    
    regions = regionprops(superpixels)
    centroids = np.empty((len(regions), 2))
    for i, region in enumerate(regions):
        center_y, center_x = region.centroid  # (y, x)
        center_x = int(round(center_x))
        center_y = int(round(center_y))
        centroids[i, 0] = center_x
        centroids[i, 1] = center_y
    
    features = feature_extractor_for_tissue.process(image, superpixels)
    
    graph = tissue_graph_builder.process(superpixels, features)
    
    canvas = visualizer.process(image, graph, instance_map=superpixels)

    return superpixels,centroids,features,canvas

def return_tissue_graph_3(image):
    
    image = np.array(image)
    
    superpixels, _ = superpixel_detector_3.process(image)
    
    regions = regionprops(superpixels)
    centroids = np.empty((len(regions), 2))
    for i, region in enumerate(regions):
        center_y, center_x = region.centroid  # (y, x)
        center_x = int(round(center_x))
        center_y = int(round(center_y))
        centroids[i, 0] = center_x
        centroids[i, 1] = center_y
    
    features = feature_extractor_for_tissue.process(image, superpixels)
    
    graph = tissue_graph_builder.process(superpixels, features)
   
    canvas = visualizer.process(image, graph, instance_map=superpixels)

    return superpixels,centroids,features,canvas

In [6]:
from histocartography.preprocessing import (
    VahadaneStainNormalizer,         # stain normalizer
    NucleiExtractor,                 # nuclei detector 
    DeepFeatureExtractor,            # feature extractor 
    KNNGraphBuilder,                 # kNN graph builder
    ColorMergedSuperpixelExtractor,  # tissue detector
    DeepFeatureExtractor,            # feature extractor
    RAGGraphBuilder,                 # build graph
    AssignmnentMatrixBuilder         # assignment matrix 
)
assignment_matrix_builder = AssignmnentMatrixBuilder()
def return_assignment_matrix(nuclei_centroid,tissue_map):
    assignment_matrix = assignment_matrix_builder.process(nuclei_centroid, tissue_map)
    return assignment_matrix

In [7]:
# test for one picture
image_path = "../image_name.png"
image = Image.open(image_path).convert("RGB")

In [8]:
nuclei_map, nuclei_centroids, features ,canvas = return_cell_graph(image)
superpixels_1, centroids_tissue_1, tissue_features_1 ,tissue_canvas_1 = return_tissue_graph_1(image)
superpixels_2, centroids_tissue_2, tissue_features_2 ,tissue_canvas_2 = return_tissue_graph_2(image)
superpixels_3, centroids_tissue_3, tissue_features_3 ,tissue_canvas_3 = return_tissue_graph_3(image)
assignment_matrix_1 = return_assignment_matrix(nuclei_centroids,superpixels_1)
assignment_matrix_2 = return_assignment_matrix(nuclei_centroids,superpixels_2)
assignment_matrix_3 = return_assignment_matrix(nuclei_centroids,superpixels_3)

Patch-level nuclei detection: 100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.73it/s]


In [9]:
pyg_data = Data(
    x_cell = features,
    x_tissue_1 = tissue_features_1,
    x_tissue_2 = tissue_features_2,
    x_tissue_3 = tissue_features_3,
    assignment_matrix_1 = torch.tensor(assignment_matrix_1),
    assignment_matrix_2 = torch.tensor(assignment_matrix_2),
    assignment_matrix_3 = torch.tensor(assignment_matrix_3),
    centroids_cell = torch.tensor(nuclei_centroids).to(torch.float32),
    centroids_tissue_1 = torch.tensor(centroids_tissue_1).to(torch.float32),
    centroids_tissue_2 = torch.tensor(centroids_tissue_2).to(torch.float32),
    centroids_tissue_3 = torch.tensor(centroids_tissue_3).to(torch.float32),
    label = 2
)

In [10]:
pyg_data

Data(assignment_matrix_1=[193, 66], assignment_matrix_2=[193, 142], assignment_matrix_3=[193, 207], centroids_cell=[193, 2], centroids_tissue_1=[66, 2], centroids_tissue_2=[142, 2], centroids_tissue_3=[207, 2], label=2, x_cell=[193, 512], x_tissue_1=[66, 512], x_tissue_2=[142, 512], x_tissue_3=[207, 512])

# Encapsulate BRACS dataset

In [11]:
nuclei_detector = NucleiExtractor()
feature_extractor_for_cell = DeepFeatureExtractor(
        architecture='resnet34',
        patch_size=72,
        resize_size=224
    )
knn_graph_builder = KNNGraphBuilder(k=5, thresh=50, add_loc_feats=True)

File already downloaded.


In [12]:
def return_cell_graph(image):
 
    image = np.array(image)
    nuclei_map, nuclei_centroids = nuclei_detector.process(image)
    
    image_size = (nuclei_map.shape[1], nuclei_map.shape[0])  # (x, y)

    features = feature_extractor_for_cell.process(image, nuclei_map)
    
    graph = knn_graph_builder.process(nuclei_map, features)
    
    canvas = visualizer.process(image, graph, instance_map=nuclei_map)   # image对象
#     canvas.show()
    
    return nuclei_map, nuclei_centroids, features,canvas

In [13]:
superpixel_detector_1 = ColorMergedSuperpixelExtractor(
        nr_superpixels=100,
        compactness=20,
        blur_kernel_size=1,
        threshold=0.001,
    )
superpixel_detector_2 = ColorMergedSuperpixelExtractor(
        nr_superpixels=400,
        compactness=20,
        blur_kernel_size=1,
        threshold=0.001,
    )
superpixel_detector_3 = ColorMergedSuperpixelExtractor(
        nr_superpixels=700,
        compactness=20,
        blur_kernel_size=1,
        threshold=0.001,
    )

feature_extractor_for_tissue = DeepFeatureExtractor(
        architecture='resnet34',
        patch_size=144,
        resize_size=224
    )
tissue_graph_builder = RAGGraphBuilder(add_loc_feats=True)

In [14]:
def return_tissue_graph_1(image):
    
    image = np.array(image)
    
    superpixels, _ = superpixel_detector_1.process(image)
    
    regions = regionprops(superpixels)
    centroids = np.empty((len(regions), 2))
    for i, region in enumerate(regions):
        center_y, center_x = region.centroid  # (y, x)
        center_x = int(round(center_x))
        center_y = int(round(center_y))
        centroids[i, 0] = center_x
        centroids[i, 1] = center_y
    
    features = feature_extractor_for_tissue.process(image, superpixels)
    
    graph = tissue_graph_builder.process(superpixels, features)
    
    canvas = visualizer.process(image, graph, instance_map=superpixels)
#     canvas.show()
    return superpixels,centroids,features,canvas


def return_tissue_graph_2(image):
    image = np.array(image)
    
    superpixels, _ = superpixel_detector_2.process(image)
    
    regions = regionprops(superpixels)
    centroids = np.empty((len(regions), 2))
    for i, region in enumerate(regions):
        center_y, center_x = region.centroid  # (y, x)
        center_x = int(round(center_x))
        center_y = int(round(center_y))
        centroids[i, 0] = center_x
        centroids[i, 1] = center_y
    
    features = feature_extractor_for_tissue.process(image, superpixels)
    
    graph = tissue_graph_builder.process(superpixels, features)
    
    canvas = visualizer.process(image, graph, instance_map=superpixels)

    return superpixels,centroids,features,canvas

def return_tissue_graph_3(image):
    
    image = np.array(image)
    
    superpixels, _ = superpixel_detector_3.process(image)
    
    regions = regionprops(superpixels)
    centroids = np.empty((len(regions), 2))
    for i, region in enumerate(regions):
        center_y, center_x = region.centroid  # (y, x)
        center_x = int(round(center_x))
        center_y = int(round(center_y))
        centroids[i, 0] = center_x
        centroids[i, 1] = center_y
    
    features = feature_extractor_for_tissue.process(image, superpixels)
    
    graph = tissue_graph_builder.process(superpixels, features)
   
    canvas = visualizer.process(image, graph, instance_map=superpixels)

    return superpixels,centroids,features,canvas

In [15]:
from histocartography.preprocessing import (
    VahadaneStainNormalizer,         # stain normalizer
    NucleiExtractor,                 # nuclei detector 
    DeepFeatureExtractor,            # feature extractor 
    KNNGraphBuilder,                 # kNN graph builder
    ColorMergedSuperpixelExtractor,  # tissue detector
    DeepFeatureExtractor,            # feature extractor
    RAGGraphBuilder,                 # build graph
    AssignmnentMatrixBuilder         # assignment matrix 
)
assignment_matrix_builder = AssignmnentMatrixBuilder()
def return_assignment_matrix(nuclei_centroid,tissue_map):
    assignment_matrix = assignment_matrix_builder.process(nuclei_centroid, tissue_map)
    return assignment_matrix

In [16]:
# test for one picture
image_path = "../BRACS_1231_N_28.png"
image = Image.open(image_path).convert("RGB")

In [17]:
nuclei_map, nuclei_centroids, features ,canvas = return_cell_graph(image)
superpixels_1, centroids_tissue_1, tissue_features_1 ,tissue_canvas_1 = return_tissue_graph_1(image)
superpixels_2, centroids_tissue_2, tissue_features_2 ,tissue_canvas_2 = return_tissue_graph_2(image)
superpixels_3, centroids_tissue_3, tissue_features_3 ,tissue_canvas_3 = return_tissue_graph_3(image)
assignment_matrix_1 = return_assignment_matrix(nuclei_centroids,superpixels_1)
assignment_matrix_2 = return_assignment_matrix(nuclei_centroids,superpixels_2)
assignment_matrix_3 = return_assignment_matrix(nuclei_centroids,superpixels_3)

Patch-level nuclei detection: 100%|█████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.20it/s]


In [18]:
pyg_data = Data(
    x_cell = features,
    x_tissue_1 = tissue_features_1,
    x_tissue_2 = tissue_features_2,
    x_tissue_3 = tissue_features_3,
    assignment_matrix_1 = torch.tensor(assignment_matrix_1),
    assignment_matrix_2 = torch.tensor(assignment_matrix_2),
    assignment_matrix_3 = torch.tensor(assignment_matrix_3),
    centroids_cell = torch.tensor(nuclei_centroids).to(torch.float32),
    centroids_tissue_1 = torch.tensor(centroids_tissue_1).to(torch.float32),
    centroids_tissue_2 = torch.tensor(centroids_tissue_2).to(torch.float32),
    centroids_tissue_3 = torch.tensor(centroids_tissue_3).to(torch.float32),
    label = 0
)

In [19]:
pyg_data

Data(assignment_matrix_1=[597, 68], assignment_matrix_2=[597, 323], assignment_matrix_3=[597, 590], centroids_cell=[597, 2], centroids_tissue_1=[68, 2], centroids_tissue_2=[323, 2], centroids_tissue_3=[590, 2], label=0, x_cell=[597, 512], x_tissue_1=[68, 512], x_tissue_2=[323, 512], x_tissue_3=[590, 512])